<a href="https://colab.research.google.com/github/NamanT98/gnn-classification/blob/main/node_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=e31a76e2ba4c854667570dcc470165b349fc23d5d4ed982ef6c6b8a917f8f133
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
import torch
import torch_geometric
torch.__version__

'2.0.1+cu118'

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [ ]:
data=Planetoid(root='.',name='cora',transform=NormalizeFeatures())

Processing...
Done!


In [ ]:
print("data type",type(data))
# print("data shape",)
print("data features",data.num_features)
print("data target",data.num_classes)
print("data length",data.len())
print("data sample",data[0])
print("sample nodes",data[0].num_nodes)
print("sample edges",data[0].num_edges)

data type <class 'torch_geometric.datasets.planetoid.Planetoid'>
data features 1433
data target 7
data length 1
data sample Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
sample nodes 2708
sample edges 10556


In [ ]:
dataset=data[0]

In [ ]:
print(dataset.x.shape)
dataset.x[0][:100]

torch.Size([2708, 1433])


tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])

In [ ]:
dataset.y

tensor([3, 4, 4,  ..., 3, 3, 3])

In [ ]:
dataset.test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [ ]:
import torch
from torch.nn import Sigmoid,Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp


class GCN(torch.nn.Module):
    def __init__(self,hidden_channels):
        super(GCN,self).__init__()
        torch.manual_seed(42)

        self.conv1=GCNConv(data.num_features,hidden_channels)
        self.conv2=GCNConv(hidden_channels,hidden_channels)
        self.out=Linear(hidden_channels,data.num_classes)

    def forward(self,x,edge_index):
        x=self.conv1(x,edge_index)
        x=x.relu()
        x=F.dropout(x,p=0.3,training=self.training)

        x=self.conv2(x,edge_index)
        x=x.relu()
        x=F.dropout(x,p=0.5,training=self.training)

        # x=torch.cat([gmp(x,batch_index),
        #                  gap(x,batch_index)],dim=1)

        x=F.softmax(self.out(x),dim=1)

        return x

model=GCN(32)
print(model)

GCN(
  (conv1): GCNConv(1433, 32)
  (conv2): GCNConv(32, 32)
  (out): Linear(in_features=32, out_features=7, bias=True)
)


In [ ]:
model=GCN(32)
print(model)

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model=model.to(device)
dataset=dataset.to(device)

learning_rate=0.001
decay=5e-4
optimizer=torch.optim.Adam(model.parameters(),
                           lr=learning_rate,
                           weight_decay=decay)


criterion=torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out=model(dataset.x,dataset.edge_index)
    loss=criterion(out[dataset.train_mask],dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out=model(dataset.x,dataset.edge_index)
    pred=out.argmax(dim=1)
    test_correct=pred[data.test_mask]==data.y[data.test_mask]
    test_acc=int(test_correct.sum())/int(data.test_mask.sum())
    return test_acc


losses=[]
for epoch in range(0,2001):
    loss=train()
    losses.append(loss)
    if epoch%100==0:
        print(f"Epoch {epoch}, Loss:{loss:4f}")

GCN(
  (conv1): GCNConv(1433, 32)
  (conv2): GCNConv(32, 32)
  (out): Linear(in_features=32, out_features=7, bias=True)
)
cpu
Epoch 0, Loss:1.946042
Epoch 100, Loss:1.944958
Epoch 200, Loss:1.928538
Epoch 300, Loss:1.797605
Epoch 400, Loss:1.690375
Epoch 500, Loss:1.582045
Epoch 600, Loss:1.512903
Epoch 700, Loss:1.479246
Epoch 800, Loss:1.405579
Epoch 900, Loss:1.401900
Epoch 1000, Loss:1.366236
Epoch 1100, Loss:1.356250
Epoch 1200, Loss:1.345531
Epoch 1300, Loss:1.309693
Epoch 1400, Loss:1.277630
Epoch 1500, Loss:1.289750
Epoch 1600, Loss:1.271060
Epoch 1700, Loss:1.260960
Epoch 1800, Loss:1.250072
Epoch 1900, Loss:1.263961
Epoch 2000, Loss:1.248340


In [ ]:
test()

0.732